Combine Data into a single dataframe and pickle it

In [4]:
import numpy as np
import pandas as pd
import os
import pickle

In [3]:
os.getcwd()

'C:\\Users\\schro\\Miniconda3\\envs\\pymc-env\\monkey-psychophysics'

In [96]:
directory_path = 'Rough_Drafts\\Data\\Sirius_data'
drop_abort_trials = True
drop_freq_manip = True
drop_repeats = True
df = pd.DataFrame()

Combine excel files into one dataframe

In [97]:
for filename in os.listdir(directory_path):
    if filename.endswith('.xlsx'):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Read the Excel file into a temporary DataFrame
        dftemp = pd.read_excel(file_path, engine='openpyxl')
        sess_date = filename[8:13]
        dftemp['sess_date'] = sess_date
        # Concatenate the temporary DataFrame with the main DataFrame
        df = pd.concat([df, dftemp], axis=0)

Drop Trials:

In [98]:
if drop_abort_trials == True:
    # drop aborted trials
    df = df[df['outcomeID'].isin([10, 31, 32, 33, 34])]

if drop_freq_manip == True:
    # drop trials with frequency manipulation
    df = df[df['tact2_Left_FR'].isin([0, 200])]
    df = df[df['tact2_Right_FR'].isin([0, 200])]
    
if drop_repeats == True:
    # drop repeated trials
    df = df[(df['repeatedTrial'] == False)]

In [99]:
df['side_idx'] = (df['tact1_Right_AMP']>0).astype(int)
df['side'] = np.where(df['tact1_Right_AMP'] > 0, 'right', 'left')

In [100]:
df['dist_idx'] = (df['tact2_Left_AMP']*df['tact2_Right_AMP']>0).astype(int)
df['dist_stat'] =  np.where(df['dist_idx'] > 0, 'bi', 'uni')

In [101]:
df['stim_amp'] = np.where(
    df['side'] == 'left', 
    df['tact2_Left_AMP'], 
    df['tact2_Right_AMP']
)

df['dist_amp'] = np.where(
    df['side'] == 'left', 
    df['tact2_Right_AMP'], 
    df['tact2_Left_AMP']
)

In [102]:
df['correct'] = (df['outcomeID']==10)

In [103]:
df['class'] = np.where(df['stim_amp'] > 28, True, False)

In [104]:
df['response'] = (~(df['correct']^df['class'])).astype(int)

In [105]:
df['group'] = df['side'] +'_'+ df['dist_stat']


In [106]:
df['group_idx'] = df['group'].map({'right_bi':3, 'right_uni':2, 'left_bi':1, 'left_uni':0})

In [107]:
group_list = ['left_uni','left_bi','right_uni','right_bi']

In [110]:
# get unique session dates in order of appearance
unique_dates = df['sess_date'].unique()
# make mapping dict
date_to_idx = {date: i for i, date in enumerate(unique_dates)}
# apply mapping
df['sess_idx'] = df['sess_date'].map(date_to_idx)
dates_list = unique_dates.astype(list)

In [114]:
sess_sum = df[['sess_date','sess_idx','dist_amp']].drop_duplicates()

In [115]:
sess_sum = sess_sum[ sess_sum['dist_amp']>0 ]

In [117]:
crosstab_counts = pd.crosstab(index=df['sess_date'], columns=df['group'])

In [121]:
sess_sum = sess_sum.set_index('sess_date')

KeyError: "None of ['sess_date'] are in the columns"

In [122]:
sess_sum = pd.concat([sess_sum, crosstab_counts], axis=1)

In [124]:
sess_sum = sess_sum.rename(columns={'left_uni':'left_uni_trial_counts','left_bi':'left_bi_trial_counts','right_uni':'right_uni_trial_counts','right_bi':'right_bi_trial_counts'})
sess_sum

,sess_idx,dist_amp,left_bi_trial_counts,left_uni_trial_counts,right_bi_trial_counts,right_uni_trial_counts
sess_date,,,,,,
03-30,0,6,77,78,76,80
03-31,1,6,77,79,79,79
04-03,2,6,68,67,66,65
04-04,3,6,80,80,80,79
04-05,4,6,79,79,80,78
04-06,5,6,80,80,80,80
04-07,6,6,69,67,70,69
04-10,7,6,46,42,44,45
04-11,8,12,80,80,80,80


In [125]:
sess_sum['trial_counts'] = sess_sum[['left_uni_trial_counts','left_bi_trial_counts','right_uni_trial_counts','right_bi_trial_counts']].sum(axis=1)

In [126]:
sess_sum['left_counts'] = sess_sum[['left_uni_trial_counts','left_bi_trial_counts']].sum(axis=1)
sess_sum['right_counts'] = sess_sum[['right_uni_trial_counts','right_bi_trial_counts']].sum(axis=1)

In [127]:
sess_sum

,sess_idx,dist_amp,left_bi_trial_counts,left_uni_trial_counts,right_bi_trial_counts,right_uni_trial_counts,trial_counts,left_counts,right_counts
sess_date,,,,,,,,,
03-30,0,6,77,78,76,80,311,155,156
03-31,1,6,77,79,79,79,314,156,158
04-03,2,6,68,67,66,65,266,135,131
04-04,3,6,80,80,80,79,319,160,159
04-05,4,6,79,79,80,78,316,158,158
04-06,5,6,80,80,80,80,320,160,160
04-07,6,6,69,67,70,69,275,136,139
04-10,7,6,46,42,44,45,177,88,89
04-11,8,12,80,80,80,80,320,160,160


In [128]:
df.head()

,Unnamed: 0,start_time,stop_time,roundID,trialID,trialCond,repeatedTrial,outcomeID,outcomeDescription,outcomeTimeStamp,fix_FixTarget_Start,fixMode,fixTarget_Window,fixTarget_Shape,fixTarget_Size,fixTarget_RGBA,fixTarget_Position,nTactile,tact1_Left_ON,tact1_Left_OFF,tact1_Left_FR,tact1_Left_AMP,tact1_Left_DUR,tact1_Right_ON,tact1_Right_OFF,tact1_Right_FR,tact1_Right_AMP,tact1_Right_DUR,tact2_Left_ON,tact2_Left_OFF,tact2_Left_FR,tact2_Left_AMP,tact2_Left_DUR,tact2_Right_ON,tact2_Right_OFF,tact2_Right_FR,tact2_Right_AMP,tact2_Right_DUR,leftPlacement,leftIndentation,rightPlacement,rightIndentation,choiceTarget_ON,fixTarget_OFF,responseMode,correctChoiceTarget_ID,selectedChoiceTarget_ID,showingCorrectChoiceTarget,nChoiceTargets,ChoiceTargets_Window,choice1_ID,choice1_Shape,choice1_Size,choice1_RGBA,choice1_Position,choice2_ID,choice2_Shape,choice2_Size,choice2_RGBA,choice2_Position,responseMovStart,fixChoiceTargetStart,rewardStart,rewardType,trialEnd,sess_date,side_idx,side,dist_idx,dist_stat,stim_amp,dist_amp,correct,class,response,group,group_idx,sess_idx
0,0,0.000,19.020,1,1,59,False,33,incorrect choiceTarget 3,7.181,0.053,eyeNoPostchoice-foot,"[9.0, 9.0]",Circle,"[1.5, 1.5]","[1.0, 1.0, 1.0, 1.0]","[0.0, 0.0]",2,-1.000,-1.000,0,0,0.0,1.785,1.885,200,28,0.1,3.423,3.923,200,6,0.5,3.423,3.923,200,18,0.5,Left Finger 3 - Distal,1000,Right Finger 3 - Distal,1000,5.455,5.471,foot,2,3,False,2,"[8.0, 8.0]",2,Triangle,"[3.5, 3.5]","[0.0, 1.0, 0.0, 1.0]","[14.0, 0.0]",3,Triangle,"[3.5, 3.5]","[0.0, 1.0, 1.0, 1.0]","[-14.0, 0.0]",6.652,6.668,-1.000,NaN,8.188,03-30,1,right,1,bi,18,6,False,False,1,right_bi,3,0
6,6,48.535,57.151,1,2,7,False,10,correct,56.137,48.999,eyeNoPostchoice-foot,"[9.0, 9.0]",Circle,"[1.5, 1.5]","[1.0, 1.0, 1.0, 1.0]","[0.0, 0.0]",2,-1.000,-1.000,0,0,0.0,50.987,51.087,200,28,0.1,-1.000,-1.000,0,0,0.0,52.620,53.120,200,32,0.5,Left Finger 3 - Distal,1000,Right Finger 3 - Distal,1000,54.642,54.653,foot,4,4,False,2,"[8.0, 8.0]",1,Triangle,"[3.5, 3.5]","[0.0, 1.0, 0.0, 1.0]","[-14.0, 0.0]",4,Triangle,"[3.5, 3.5]","[0.0, 1.0, 1.0, 1.0]","[14.0, 0.0]",55.267,55.290,55.804,juice,56.137,03-30,1,right,0,uni,32,0,True,True,1,right_uni,2,0
7,7,57.152,65.303,1,3,79,False,10,correct,64.285,57.217,eyeNoPostchoice-foot,"[9.0, 9.0]",Circle,"[1.5, 1.5]","[1.0, 1.0, 1.0, 1.0]","[0.0, 0.0]",2,58.737,58.837,200,28,0.1,-1.000,-1.000,0,0,0.0,60.370,60.870,200,44,0.5,60.370,60.870,200,6,0.5,Left Finger 3 - Distal,1000,Right Finger 3 - Distal,1000,62.403,62.420,foot,3,3,False,2,"[8.0, 8.0]",2,Triangle,"[3.5, 3.5]","[0.0, 1.0, 0.0, 1.0]","[14.0, 0.0]",3,Triangle,"[3.5, 3.5]","[0.0, 1.0, 1.0, 1.0]","[-14.0, 0.0]",63.438,63.451,63.967,juice,64.285,03-30,0,left,1,bi,44,6,True,True,1,left_bi,1,0
8,8,65.304,73.316,1,4,15,False,10,correct,72.287,65.616,eyeNoPostchoice-foot,"[9.0, 9.0]",Circle,"[1.5, 1.5]","[1.0, 1.0, 1.0, 1.0]","[0.0, 0.0]",2,-1.000,-1.000,0,0,0.0,67.187,67.287,200,28,0.1,-1.000,-1.000,0,0,0.0,68.820,69.320,200,6,0.5,Left Finger 3 - Distal,1000,Right Finger 3 - Distal,1000,70.853,70.869,foot,2,2,False,2,"[8.0, 8.0]",2,Triangle,"[3.5, 3.5]","[0.0, 1.0, 0.0, 1.0]","[14.0, 0.0]",3,Triangle,"[3.5, 3.5]","[0.0, 1.0, 1.0, 1.0]","[-14.0, 0.0]",71.432,71.449,71.953,juice,72.287,03-30,1,right,0,uni,6,0,True,False,0,right_uni,2,0
11,11,83.269,91.290,1,7,41,False,10,correct,90.270,83.320,eyeNoPostchoice-foot,"[9.0, 9.0]",Circle,"[1.5, 1.5]","[1.0, 1.0, 1.0, 1.0]","[0.0, 0.0]",2,85.258,85.358,200,28,0.1,-1.000,-1.000,0,0,0.0,86.883,87.383,200,18,0.5,-1.000,-1.000,0,0,0.0,Left Finger 3 - Distal,1000,Right Finger 3 - Distal,1000,88.907,88.924,foot,2,2,False,2,"[8.0, 8.0]",2,Triangle,"[3.5, 3.5]","[0.0, 1.0, 0.0, 1.0]","[14.0, 0.0]",3,Triangle,"[3.5, 3.5]","[0.0, 1.0, 1.0, 1.0]","[-14.0, 0.0]",89.416,89.433,89.941,juice,90.270,03-30,0,left,0,uni,18,0,True,False,0,left_uni,0,0


In [130]:
DataDict = {'session_summary': sess_sum, 'data':  df}

In [131]:
with open("Sirius_Data.pkl","wb") as f:
    pickle.dump(DataDict, f)